## ddqc Tutorial
First, lets set up the workspace loading the ddqc and pegasus packages

In [1]:
import pegasus as pg
import ddqc

### Reading the data
For this tutorial we will use the Human Bone Marrow dataset provided by pegasus. <br>
We will download the dataset using the following command:

In [ ]:
!curl https://storage.googleapis.com/terra-featured-workspaces/Cumulus/MantonBM_nonmix_subset.zarr.zip -o MantonBM_nonmix_subset.zarr.zip

We will first load the data using the standard `read_input` pegasus function.

In [2]:
data = pg.read_input("MantonBM_nonmix_subset.zarr.zip")
data

2022-01-16 21:03:33,262 - pegasusio.readwrite - INFO - zarr file 'MantonBM_nonmix_subset.zarr.zip' is loaded.
2022-01-16 21:03:33,263 - pegasusio.readwrite - INFO - Function 'read_input' finished in 0.24s.


MultimodalData object with 1 UnimodalData: 'GRCh38-rna'
    It currently binds to UnimodalData object GRCh38-rna

UnimodalData object with n_obs x n_vars = 48219 x 36601
    Genome: GRCh38; Modality: rna
    It contains 1 matrix: 'X'
    It currently binds to matrix 'X' as X

    obs: 'n_genes', 'Channel'
    var: 'featureid'
    uns: 'genome', 'modality'

### Biology-centered data-driven quality control (ddqc)
ddqc does adaptive quality control by clustering cells and picking a separate threshold for each cluster. It is described down below:
- Initial Qualtity Control (QC) is performed, when obvious low-quality cells are removed. By default those are cells with n_genes < 100 and percent_mito > 80.
- The cells are clustered with the clustering resolution 1.3 (default)
- Then thesholds are picked for each cluster. By default the following metrics are considered:
    - **Number of counts/UMIs**: keep cells that have n_counts greater than median - 2 Median Absolute Deviations (MAD)
    - **Number of genes**: keep cells that have n_genes greater than median - 2 MADs
    - **Percent of mitochondrial transctipts**: keep cells that have percent_mito less than median + 2 MADs
- In order to prevent the removal of healthy cells in clusters with high median n_genes and low percent_mito there are additional bounds for those thresholds:
    - Cluster-level threshold for n_genes can't be greater than 200 (default). If it is greater, it will be set to 200.
    - Cluster-level threshold for percent_mito can't be lower than 10 (default). If it is lower, it will be set to 10.

To perform ddqc on a dataset, we will run the `ddqc_metrics` function. 

If you want to customize the filtering you can use the following parameters:
- `res`: float - clustering resolution (default: 1.3)
- `clustering_method`: str - clustering method to use (default: "louvain", available options "louvain", "leiden", "spectral_louvain", "spectral_leiden")
- `n_components` - number of PCA components (default: 50)
- `K` - k to be used by `neighbors` Pegasus function (default: 20)
- `method`: string - statistic on which the threshold would be calculated (default: "mad", available options "mad", "outlier")
- `threshold`: float - parameter for the selected method (default: 2)
- `basic_n_genes`: int - parameter for the initial QC n_genes filtering (default: 100)
- `basic_percent_mito`: float - parameter for the initial QC percent_mito filtering (default: 80)
- `mito_prefix`: string - gene prefix used to calculate percent_mito in a cell (default: "MT-")
- `ribo_prefix`: string - gene regular expression used to calculate percent_ribo in a cell (default: "^RP[SL][[:digit:]]|^RPLP[[:digit:]]|^RPSA")
- `do_counts`: bool - whether to consider n_counts for ddqc (default: True)
- `do_genes`: bool - whether to consider n_genes for ddqc (default: True)
- `do_mito`: bool - whether to consider percent_mito for ddqc (default: True)
- `do_ribo`: bool - whether to consider percent_ribo for ddqc (default: False)
- `n_genes_lower_bound`: int - bound for lower n_genes cluster-level threshold (default: 200)
- `percent_mito_upper_bound`: float - bound for upper percent_mito cluster-level threshold (default: 10)
- `random_state`: int - random seed for clustering results reproducibility (default: 29)
- `return_df_qc`: bool - whether to return a dataframe with the information about on what metric and what threshold the cell was removed for each removed cell. (default: True)

We will now run ddqc on the dataset with default settings.

In [4]:
df_qc = ddqc.ddqc_metrics(data, return_df_qc=True)

0 0 0 spectral_leiden
2022-01-16 21:04:27,727 - pegasusio.qc_utils - INFO - After filtration, 48210 out of 48210 cell barcodes are kept in UnimodalData object GRCh38-rna.
2022-01-16 21:04:27,729 - pegasus.tools.preprocessing - INFO - Function 'filter_data' finished in 0.18s.
2022-01-16 21:04:28,068 - pegasus.tools.preprocessing - INFO - After filtration, 25910/25910 genes are kept. Among 25910 genes, 16967 genes are robust.
2022-01-16 21:04:28,069 - pegasus.tools.preprocessing - INFO - Function 'identify_robust_genes' finished in 0.34s.
2022-01-16 21:04:28,759 - pegasus.tools.preprocessing - INFO - Function 'log_norm' finished in 0.68s.
2022-01-16 21:04:28,884 - pegasus.tools.hvf_selection - INFO - Function 'estimate_feature_statistics' finished in 0.12s.
2022-01-16 21:04:28,917 - pegasus.tools.hvf_selection - INFO - 2000 highly variable features have been selected.
2022-01-16 21:04:28,918 - pegasus.tools.hvf_selection - INFO - Function 'highly_variable_features' finished in 0.16s.
202

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_methods.py:180: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


2022-01-16 21:05:14,998 - pegasus.tools.nearest_neighbors - INFO - Function 'calculate_affinity_matrix' finished in 0.64s.
2022-01-16 21:05:15,005 - pegasus.tools.clustering - WARNING - diffmap is not calculated, switch to pca instead.


ValueError: Found array with 0 feature(s) (shape=(48210, 0)) while a minimum of 1 is required.

### Outputs
There are two boxplots provided for exploratory data analysis:
- log2(n_genes) by cluster: shows log2 of number of genes for each cluster in the initial clustering. Red line at 200 genes (7.64 in log2 scale) represents the most common fixed threshold cutoff for n_genes.
- percent_mito by cluster: shows percent_mito for each cluster in the initial clustering. Red line at 10% represents the most common fixed threshold cutoff for percent_mito.

If you requested to return df_qc the function will return a pandas dataframe containing the following info for each cell:
- `metric`: QC metric number
- `cluster_labels`: cluster from initial clustering performed by ddqc
- `metric_lower_co` and `metric_upper_co`: lower and upper cuttofs for each metric on which ddqc was performed. If ddqc was not performed for upper or lower end of this metric this field will be `None`
- `metric.passed.qc`: whether the cell passed qc for a given metric
This information is useful if you want to understand based on which metric the cell was filtered out.

In [ ]:
df_qc

### Filter out the cells
Now, we will filter out the cells that failed ddqc. Here, we started with 48219 cells and retain 45939 cells.

In [ ]:
pg.filter_data(data)

### Save the dataset
We will save the dataset as data.h5ad, which can then be imported into the scRNAseq pipeline of your choice for downstream analysis.

In [ ]:
pg.write_output(data, "data.h5ad")